In [2]:
import os
import gc
import numpy as np
import cv2 as cv
from tqdm import tqdm
import common as cmn

In [6]:
def Open(path):
    img = cv.imread(path)
    return img


def Resize(img, size):
    initH = img.shape[0]
    initW = img.shape[1]

    reqH = size[0]
    reqW = size[1]

    if (reqH == initH and reqW == initW):
        return img

    if reqH/initH > reqW/initW:
        actH = reqH
        hCrop = 0

        actW = initW * reqH//initH
        wCrop = abs(actW - reqW)//2
    else:
        actW = reqW
        wCrop = 0

        actH = initH * reqW//initW
        hCrop = abs(actH - reqH)//2

    resized = cv.resize(img, (actW, actH), interpolation=cv.INTER_AREA)
    cropped = resized[hCrop: hCrop + reqH, wCrop: wCrop + reqW, :]

    return cropped


def Save(img, path):
    cv.imwrite(path, img)


def ProcessImage(inPath, outPath, kwargs):
    img = Open(inPath)
    img = Resize(img, kwargs.get("size"))
    Save(img, outPath)


def ProcessFolder(inFolderPath, outFolderPath, kwargs, fraction=None, balance=False):
    folderInfo = cmn.get_folder_info(inFolderPath)

    for fi in tqdm(folderInfo, desc="Folders", leave=True):

        outPath = os.path.join(outFolderPath, cmn.get_class(fi[0]))
        if not os.path.exists(outPath):
            os.makedirs(outPath)

        if balance:
            takeN = min([fi[1] for fi in folderInfo])
        else:
            takeN = fi[1]

        if fraction != None:
            takeN = int(fraction * takeN)

        if takeN == fi[1]:
            picsToProcess = fi[2]
        else:
            picsToProcess = np.random.choice(fi[2], takeN, replace=False)

        for pic in tqdm(picsToProcess, desc=outPath, leave=True):
            inPath = os.path.join(fi[0], pic)
            newPath = os.path.join(outPath, pic)
            ProcessImage(inPath, newPath, kwargs)

    gc.collect()


In [8]:
flatPath = "../Data/Flat"

# AlexNet
# prePath = "../Data/PreprocessedAlexNet"
# size = (227, 227) # AlexNet

# ResNet50, VGG16
prePath = "../Data/PreprocessedResNet"
size = (224, 224) 

cmn.clean_folder(prePath)
ProcessFolder(flatPath, prePath, kwargs={"size": size}, fraction=0.1, balance=True)

Folders: 0it [00:00, ?it/s]
Folders: 100%|██████████| 2/2 [00:51<00:00, 25.70s/it]
